<a href="https://colab.research.google.com/github/marcelo7bastos/mba_enap_introducao_ciencia_dados/blob/main/trabalhos_realizados/Aula8_ContarDosesCovid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contar doses de vacinas da COVID-19

In [21]:
import pandas as pd
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
## Defina o id do seu projeto no bigquery!!!!!
project_id = '365846072239' # Defina o id do seu projeto no bigquery!!!!!
## Defina o id do seu projeto no bigquery!!!!!



Authenticated


### Dica: Restrinja seus testes aos estados do AC, AP e RR para diminuir o tamanho da sua Query.

In [22]:


df_vacina = pd.io.gbq.read_gbq('''
SELECT
    sigla_uf
    ,vacina
    ,count(*) as qt_total
    ,sum(case when regexp_contains(dose,'1ª Dose$|Inicial') then 1 else 0 end) as qt_D1
    ,sum(case when regexp_contains(dose,'2ª Dose$') then 1 else 0 end) as qt_D2
    ,sum(case when regexp_contains(dose,'Reforço') then 1 else 0 end) as qt_Reforco
    ,sum(case when regexp_contains(dose,'Dose Adicional|3ª Dose') then 1 else 0 end) as qt_Adicional
    ,sum(case when regexp_contains(dose,'Única|^Dose$') then 1 else 0 end) as qt_Unica
FROM `basedosdados.br_ms_vacinacao_covid19.microdados_vacinacao`
where vacina in ('85','86','87','88','89') and sigla_uf in ('AC','AP','RR')
group by sigla_uf, vacina
limit 20;
''', project_id=project_id)

df_vacina.head()

/tmp/ipython-input-3590589861.py:1: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_vacina = pd.io.gbq.read_gbq('''


,sigla_uf,vacina,qt_total,qt_D1,qt_D2,qt_Reforco,qt_Adicional,qt_Unica
0,AC,86,185531,100467,84938,109,17,0
1,AC,87,457364,243218,156853,49567,7726,0
2,AC,85,422293,222991,195394,3584,324,0
3,AC,89,25492,12102,13375,15,0,0
4,AC,88,14251,0,0,1874,150,12227


### Calcular a semana epidemiológica (domingo a sábado) a partir da data de vacinação

In [23]:
df_vacina = pd.io.gbq.read_gbq('''
SELECT
    sigla_uf
    ,(case
        when vacina='86' then 'Coronavac'
        when vacina='87' then 'Pfizer'
        when vacina='88' then 'Janssen'
        else 'Astrazeneca'
      end
    ) as vacina_apelido
    ,(case when EXTRACT(YEAR FROM data_aplicacao) >= 2021 then DATE_TRUNC(data_aplicacao, WEEK(SUNDAY)) else PARSE_DATE('%d/%m/%Y',  '03/01/2021') end) as semana
    ,count(*) as qt_total
    ,sum(case when regexp_contains(dose,'1ª Dose$|^Dose$|Inicial') then 1 else 0 end) as qt_D1
    ,sum(case when regexp_contains(dose,'2ª Dose$') then 1 else 0 end) as qt_D2
    ,sum(case when regexp_contains(dose,'Reforço') then 1 else 0 end) as qt_Reforco
    ,sum(case when regexp_contains(dose,'Dose Adicional|3ª Dose') then 1 else 0 end) as qt_Adicional
    ,sum(case when regexp_contains(dose,'Única') then 1 else 0 end) as qt_Unica
FROM `basedosdados.br_ms_vacinacao_covid19.microdados_vacinacao`
where vacina in ('85','86','87','88','89') and EXTRACT(YEAR FROM data_aplicacao) >= 2021
group by vacina_apelido, sigla_uf, semana
''', project_id=project_id)


df_vacina.to_gbq("enapdatasets.vacinacao",
  project_id=project_id,
  chunksize=40000,
  if_exists='replace',
  )
df_vacina.head()


/tmp/ipython-input-1108505673.py:1: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_vacina = pd.io.gbq.read_gbq('''
/tmp/ipython-input-1108505673.py:24: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_vacina.to_gbq("enapdatasets.vacinacao",
100%|██████████| 1/1 [00:00<00:00, 2447.09it/s]


,sigla_uf,vacina_apelido,semana,qt_total,qt_D1,qt_D2,qt_Reforco,qt_Adicional,qt_Unica
0,MT,Astrazeneca,2021-08-22,58914,9176,49738,0,0,0
1,MT,Astrazeneca,2021-11-07,21311,299,20377,627,8,0
2,MT,Janssen,2021-06-20,780,0,0,0,0,780
3,AM,Astrazeneca,2021-08-08,48325,8769,39554,2,0,0
4,AM,Janssen,2021-07-25,4907,0,0,0,9,4898


## Query para projeção de 2a dose da vacina do Covid
### Requisito de negócio: 2a dose deve ser aplicada a partir de 56 dias após a 1a dose.
#### Também seria útil conhecer a demanda com antecedência (21 dias depois de aplicada a 1a dose)

In [26]:
df_vacina_proj_teste = pd.io.gbq.read_gbq('''
    SELECT sigla_uf as sigla_uf_proj, vacina_apelido as vacina_apelido_proj, qt_D1 as qt_D2_Proj, DATE_ADD(semana, INTERVAL 21 DAY) as semana_proj, semana
    FROM `enapdatasets.vacinacao`vp
where sigla_uf='RR'
order by vp.sigla_uf, vp.semana, vp.sigla_uf, vp.vacina_apelido
''', project_id=project_id)

df_vacina_proj_teste.head(15)

/tmp/ipython-input-554294613.py:1: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_vacina_proj_teste = pd.io.gbq.read_gbq('''


,sigla_uf_proj,vacina_apelido_proj,qt_D2_Proj,semana_proj,semana
0,RR,Astrazeneca,4,2021-02-07,2021-01-17
1,RR,Coronavac,4675,2021-02-07,2021-01-17
2,RR,Pfizer,3,2021-02-07,2021-01-17
3,RR,Astrazeneca,11,2021-02-14,2021-01-24
4,RR,Coronavac,7610,2021-02-14,2021-01-24
5,RR,Pfizer,2,2021-02-14,2021-01-24
6,RR,Astrazeneca,357,2021-02-21,2021-01-31
7,RR,Coronavac,5743,2021-02-21,2021-01-31
8,RR,Astrazeneca,4,2021-02-28,2021-02-07
9,RR,Coronavac,7076,2021-02-28,2021-02-07


In [27]:
df_vacina_proj = pd.io.gbq.read_gbq('''
SELECT v.sigla_uf, v.vacina_apelido, v.semana, v.qt_total, v.qt_D1, v.qt_D2, v.qt_Reforco, v.qt_Adicional, v.qt_Unica, vp.qt_D2_Proj, vp.semana_proj, vp.sigla_uf_proj, vp. vacina_apelido_proj
FROM `enapdatasets.vacinacao` v
LEFT JOIN (
    SELECT sigla_uf as sigla_uf_proj, vacina_apelido as vacina_apelido_proj, qt_D1 as qt_D2_Proj, DATE_ADD(semana, INTERVAL 21 DAY) as semana_proj
    FROM `enapdatasets.vacinacao`
    ) as vp
ON v.sigla_uf=vp.sigla_uf_proj and v.vacina_apelido=vp.vacina_apelido_proj and vp.semana_proj=v.semana
where sigla_uf in ('AC','AP','RR')
order by v.sigla_uf, v.semana, vp.semana_proj, vp.sigla_uf_proj, vp.vacina_apelido_proj
''', project_id=project_id)

df_vacina_proj.head()

/tmp/ipython-input-1813122139.py:1: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_vacina_proj = pd.io.gbq.read_gbq('''


,sigla_uf,vacina_apelido,semana,qt_total,qt_D1,qt_D2,qt_Reforco,qt_Adicional,qt_Unica,qt_D2_Proj,semana_proj,sigla_uf_proj,vacina_apelido_proj
0,AC,Astrazeneca,2021-01-17,13,12,1,0,0,0,<NA>,NaT,None,None
1,AC,Coronavac,2021-01-17,3978,3973,5,0,0,0,<NA>,NaT,None,None
2,AC,Pfizer,2021-01-17,4,3,0,1,0,0,<NA>,NaT,None,None
3,AC,Astrazeneca,2021-01-24,794,792,2,0,0,0,<NA>,NaT,None,None
4,AC,Coronavac,2021-01-24,5962,5951,11,0,0,0,<NA>,NaT,None,None


In [28]:
# df_vacina_proj['semana_proj'].max()
df_vacina_proj.iloc[20:30]

,sigla_uf,vacina_apelido,semana,qt_total,qt_D1,qt_D2,qt_Reforco,qt_Adicional,qt_Unica,qt_D2_Proj,semana_proj,sigla_uf_proj,vacina_apelido_proj
20,AC,Coronavac,2021-02-28,4014,2593,1421,0,0,0,1882,2021-02-28,AC,Coronavac
21,AC,Pfizer,2021-02-28,2,1,1,0,0,0,3,2021-02-28,AC,Pfizer
22,AC,Astrazeneca,2021-03-07,1592,1531,61,0,0,0,747,2021-03-07,AC,Astrazeneca
23,AC,Coronavac,2021-03-07,4886,3374,1512,0,0,0,2198,2021-03-07,AC,Coronavac
24,AC,Pfizer,2021-03-07,1,1,0,0,0,0,2,2021-03-07,AC,Pfizer
25,AC,Astrazeneca,2021-03-14,562,551,11,0,0,0,5257,2021-03-14,AC,Astrazeneca
26,AC,Coronavac,2021-03-14,5832,4041,1791,0,0,0,1563,2021-03-14,AC,Coronavac
27,AC,Astrazeneca,2021-03-21,6736,6714,22,0,0,0,6778,2021-03-21,AC,Astrazeneca
28,AC,Coronavac,2021-03-21,5888,4292,1596,0,0,0,2593,2021-03-21,AC,Coronavac
29,AC,Pfizer,2021-03-21,2,2,0,0,0,0,1,2021-03-21,AC,Pfizer


## Query join com tabela de todas as combinações de vacina_apelido, sigla_uf e semana  (BigQuery)
### Dois problemas corrigidos: 1) essa tabela com todas as combinações e doses projetadas de 2020.

In [29]:
df_vacina_proj = pd.io.gbq.read_gbq('''
SELECT COALESCE(wy.week, v.semana) as semana, COALESCE(wy.sigla_uf, v.sigla_uf) as sigla_uf, COALESCE(wy.vacina_apelido, v.vacina_apelido) as vacina_apelido, COALESCE(v.qt_total,wy.qt_total) as qt_total, COALESCE(v.qt_D1,wy.qt_D1) as qt_D1, COALESCE(v.qt_D2,wy.qt_D2) as qt_D2, COALESCE(v.qt_Reforco,wy.qt_Reforco) as qt_Reforco, COALESCE(v.qt_Adicional,wy.qt_Adicional) as qt_Adicional, COALESCE(v.qt_Unica,wy.qt_Unica) as qt_Unica, COALESCE(vp.qt_D2_Proj,0) as qt_D2_Proj, vp.semana_proj, vp.sigla_uf_proj, vp.vacina_apelido_proj
FROM (
    WITH sundays AS (
  SELECT d AS week
  FROM UNNEST(GENERATE_DATE_ARRAY(DATE '2021-01-01', DATE '2022-12-31', INTERVAL 1 DAY)) d
  WHERE EXTRACT(DAYOFWEEK FROM d) = 1      -- Sundays
),
ufs AS (
  SELECT DISTINCT sigla_uf
  FROM `enapdatasets.vacinacao`
),
vacinas AS (
  SELECT vacina_apelido
  FROM UNNEST(['Coronavac','Pfizer','Janssen','Astrazeneca']) AS vacina_apelido
)
SELECT
  u.sigla_uf,  vi.vacina_apelido,  s.week,  0 AS qt_total,  0 AS qt_D1,  0 AS qt_D2,  0 AS qt_Reforco,  0 AS qt_Adicional,  0 AS qt_Unica
FROM ufs u
CROSS JOIN vacinas vi
CROSS JOIN sundays s
) wy
left join `enapdatasets.vacinacao` v on wy.week = v.semana and wy.vacina_apelido=v.vacina_apelido and wy.sigla_uf=v.sigla_uf
LEFT JOIN (
    SELECT sigla_uf as sigla_uf_proj, vacina_apelido as vacina_apelido_proj, qt_D1 as qt_D2_Proj, DATE_ADD(semana, INTERVAL @qtd_dias_proj_d2 DAY) as semana_proj
    FROM `enapdatasets.vacinacao` where EXTRACT(YEAR FROM semana) >= 2021
    ) as vp
ON v.sigla_uf=vp.sigla_uf_proj and v.vacina_apelido=vp.vacina_apelido_proj and vp.semana_proj=wy.week
order by v.sigla_uf, wy.week, vp.semana_proj, vp.sigla_uf_proj, vp.vacina_apelido_proj
''', project_id=project_id)

df_vacina_proj.head()

/tmp/ipython-input-4225572805.py:1: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_vacina_proj = pd.io.gbq.read_gbq('''


GenericGBQException: Reason: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/365846072239/queries?prettyPrint=false: Query parameter 'qtd_dias_proj_d2' not found at [25:125]

### Código com a quanitdade de dias parametrizado
```
SELECT COALESCE(wy.week, v.semana) as semana, COALESCE(wy.sigla_uf, v.sigla_uf) as sigla_uf, COALESCE(wy.vacina_apelido, v.vacina_apelido) as vacina_apelido, COALESCE(v.qt_total,wy.qt_total) as qt_total, COALESCE(v.qt_D1,wy.qt_D1) as qt_D1, COALESCE(v.qt_D2,wy.qt_D2) as qt_D2, COALESCE(v.qt_Reforco,wy.qt_Reforco) as qt_Reforco, COALESCE(v.qt_Adicional,wy.qt_Adicional) as qt_Adicional, COALESCE(v.qt_Unica,wy.qt_Unica) as qt_Unica, COALESCE(vp.qt_D2_Proj,0) as qt_D2_Proj, vp.semana_proj, vp.sigla_uf_proj, vp.vacina_apelido_proj
FROM (
    WITH sundays AS (
  SELECT d AS week
  FROM UNNEST(GENERATE_DATE_ARRAY(DATE '2021-01-01', DATE '2022-12-31', INTERVAL 1 DAY)) d
  WHERE EXTRACT(DAYOFWEEK FROM d) = 1      -- Sundays
),
ufs AS (
  SELECT DISTINCT sigla_uf
  FROM `enapdatasets.vacinacao`
),
vacinas AS (
  SELECT vacina_apelido
  FROM UNNEST(['Coronavac','Pfizer','Janssen','Astrazeneca']) AS vacina_apelido
)
SELECT
  u.sigla_uf,  vi.vacina_apelido,  s.week,  0 AS qt_total,  0 AS qt_D1,  0 AS qt_D2,  0 AS qt_Reforco,  0 AS qt_Adicional,  0 AS qt_Unica
FROM ufs u
CROSS JOIN vacinas vi
CROSS JOIN sundays s
) wy
left join `enapdatasets.vacinacao` v on wy.week = v.semana and wy.vacina_apelido=v.vacina_apelido and wy.sigla_uf=v.sigla_uf
LEFT JOIN (
    SELECT sigla_uf as sigla_uf_proj, vacina_apelido as vacina_apelido_proj, qt_D1 as qt_D2_Proj, DATE_ADD(semana, INTERVAL @qtd_dias_proj_d2 DAY) as semana_proj
    FROM `enapdatasets.vacinacao` where EXTRACT(YEAR FROM semana) >= 2021
    ) as vp
ON v.sigla_uf=vp.sigla_uf_proj and v.vacina_apelido=vp.vacina_apelido_proj and vp.semana_proj=wy.week
order by v.sigla_uf, wy.week, vp.semana_proj, vp.sigla_uf_proj, vp.vacina_apelido_proj

```